## Import Libraries

In [1]:
import pandas as pd

from scipy import stats

# Import libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import psycopg2

# below imports are used to print out pretty pandas dataframes
from IPython.display import display, HTML

%matplotlib inline
plt.style.use('ggplot')

## Connect to DB

In [2]:
# information used to create a database connection
sqluser = 'postgres'
dbname = 'mimic'
hostname = 'localhost'
port_number = 5434

# Connect to postgres with a copy of the MIMIC-III database
con = psycopg2.connect(dbname=dbname, user=sqluser, host=hostname, port=port_number, password='mysecretpassword')

# the below statement is prepended to queries to ensure they select from the right schema
# query_schema = 'set search_path to ' + schema_name + ';'

In [6]:
sepsis_query = \
"""
SELECT
DISTINCT
PAT.subject_id,
ADM.hadm_id,
ICU.icustay_id,
DATE_PART('year', AGE(ADM.admittime, PAT.dob)) as "age",
ADM.ethnicity AS "ethnicity",
ICU.first_careunit AS "first_careunit",
ADM.marital_status AS "marital_status",
ADM.insurance AS "insurance",
ADM.admission_type AS "admission_type",
ADM.admission_location AS "admission_location",
ICU_D.first_hosp_stay AS "first_hosp_stay",
ICU_D.los_hospital AS "los_hospital",
ICU_D.first_icu_stay AS "first_icu_stay",
ICU_D.los_icu AS "los_icu",
VIT.heartrate_mean AS "heartrate_mean",
VIT.sysbp_mean AS "sysbp_mean",
VIT.diasbp_mean AS "diasbp_mean",
VIT.meanbp_mean AS "meanbp_mean",
VIT.resprate_mean AS "resprate_mean",
VIT.tempc_mean AS "tempc_mean",
VIT.spo2_mean AS "spo2_mean",
VIT.glucose_mean AS "glucose_mean",
ADM.hospital_expire_flag AS "hospital_expire_flag"
FROM patients PAT
INNER JOIN admissions ADM
ON PAT.subject_id = ADM.subject_id
LEFT JOIN icustays ICU
ON ICU.subject_id = ADM.subject_id AND ICU.hadm_id = ADM.hadm_id
LEFT JOIN icustay_detail ICU_D
ON ICU_D.icustay_id = ICU.icustay_id
LEFT JOIN vitals_first_day VIT
ON VIT.subject_id = ADM.subject_id AND VIT.hadm_id = ADM.hadm_id AND VIT.icustay_id = ICU.icustay_id
INNER JOIN diagnoses_icd DIA_ICD
ON DIA_ICD.subject_id = ADM.subject_id AND DIA_ICD.hadm_id = ADM.hadm_id AND DIA_ICD.icd9_code in ('99591', '99592', '78552')
WHERE DATE_PART('year', AGE(ADM.admittime, PAT.dob)) > 18
;
"""

sepsis_patients_1 = pd.read_sql_query(sepsis_query, con)

In [ ]:
sepsis_query = \
"""
SELECT
DISTINCT
PAT.subject_id,
ADM.hadm_id,
ICU.icustay_id,
CRRT.duration_hours as "crrt_duration_hours",
PPN.duration_hours as "ppn_duration_hours",
ADE.duration_hours as "ade_duration_hours",
VPN.duration_hours as "vpn_duration_hours",
DMN.duration_hours as "dmn_duration_hours",
VPR.duration_hours as "vpr_duration_hours",
DPN.duration_hours as "dpn_duration_hours",
EPN.duration_hours as "epn_duration_hours",
ISP.duration_hours as "isp_duration_hours",
MRN.duration_hours as "mrn_duration_hours",
VLN.duration_hours as "vln_duration_hours",
NPN.duration_hours as "npn_duration_hours",
ADM.hospital_expire_flag AS "hospital_expire_flag"
FROM patients PAT
INNER JOIN admissions ADM
ON PAT.subject_id = ADM.subject_id
LEFT JOIN icustays ICU
ON ICU.subject_id = ADM.subject_id AND ICU.hadm_id = ADM.hadm_id
LEFT JOIN icustay_detail ICU_D
ON ICU_D.icustay_id = ICU.icustay_id
LEFT JOIN crrt_durations CRRT
ON ICU_D.icustay_id = CRRT.icustay_id
LEFT JOIN phenylephrine_durations PPN
ON ICU_D.icustay_id = PPN.icustay_id
LEFT JOIN adenosine_durations ADE
ON ICU_D.icustay_id = ADE.icustay_id
LEFT JOIN vasopressin_durations VPN
ON ICU_D.icustay_id = VPN.icustay_id
LEFT JOIN dobutamine_durations DMN
ON ICU_D.icustay_id = DMN.icustay_id
LEFT JOIN vasopressor_durations VPR
ON ICU_D.icustay_id = VPR.icustay_id
LEFT JOIN dopamine_durations DPN
ON ICU_D.icustay_id = DPN.icustay_id
LEFT JOIN epinephrine_durations EPN
ON ICU_D.icustay_id = EPN.icustay_id
LEFT JOIN isuprel_durations ISP
ON ICU_D.icustay_id = ISP.icustay_id
LEFT JOIN milrinone_durations MRN
ON ICU_D.icustay_id = MRN.icustay_id
LEFT JOIN ventilation_durations VLN
ON ICU_D.icustay_id = VLN.icustay_id
LEFT JOIN norepinephrine_durations NPN
ON ICU_D.icustay_id = NPN.icustay_id
INNER JOIN diagnoses_icd DIA_ICD
ON DIA_ICD.subject_id = ADM.subject_id AND DIA_ICD.hadm_id = ADM.hadm_id AND DIA_ICD.icd9_code in ('99591', '99592', '78552')
WHERE DATE_PART('year', AGE(ADM.admittime, PAT.dob)) > 18
;
"""

sepsis_patients_2 = pd.read_sql_query(sepsis_query, con)